# Deeplab v3+

This notebook shows how to train a Deeplab v3+ semantic segmentation model on a custom coco-style data set.

In [1]:
import os
import sys
import numpy as np

sys.path.insert(0, '../libraries')
from wdeeplab.config import Config
import wdeeplab.model as modellib
import wdeeplab.utils as utils
import wdataset.coco as coco

%matplotlib inline

HOME_DIR = '/home/keras'
DATA_DIR = os.path.join(HOME_DIR, "data/shapes")
MODEL_DIR = os.path.join(DATA_DIR, "logs")
WEIGHTS_DIR = os.path.join(HOME_DIR, "data")
PASCAL_WEIGHTS_PATH = os.path.join(WEIGHTS_DIR, "weights/deeplabv3_weights_tf_dim_ordering_tf_kernels.h5")
if not os.path.exists(PASCAL_WEIGHTS_PATH):
    print("Downloading PASCAL VOC trained weights to {}".format(PASCAL_WEIGHTS_PATH))
    utils.download_trained_weights(WEIGHTS_DIR)
else:
    print("PASCAL VOC trained weights already downloaded to {}".format(PASCAL_WEIGHTS_PATH))

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


PASCAL VOC trained weights already downloaded to /home/keras/data/weights/deeplabv3_weights_tf_dim_ordering_tf_kernels.h5



## Dataset

Organize the dataset using the following structure:


    DATA_DIR
    │
    └───annotations
    │   │   instances_<subset><year>.json
    │   
    └───<subset><year>
        │   image021.jpeg
        │   image022.jpeg



In [3]:
dataset_train = coco.CocoDataset()
dataset_train.load_coco(DATA_DIR, subset="shapes_train", year="2018")
dataset_train.prepare()

dataset_validate = coco.CocoDataset()
dataset_validate.load_coco(DATA_DIR, subset="shapes_validate", year="2018")
dataset_validate.prepare()

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


## Configuration

In [4]:
image_size = 64

class ShapesConfig(Config):
    """Configuration for training on the shapes dataset.
    """
    NAME = "shapes"

    # Train on 1 GPU and 1 images per GPU. Put multiple images on each
    # GPU if the images are small.
    # BATCH_SIZE = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # background + 3 shapes (triangles, circles, and squares)

    # Use smaller images for faster training. 
    IMAGE_MAX_DIM = image_size
    IMAGE_MIN_DIM = image_size

    STEPS_PER_EPOCH = 200

    VALIDATION_STEPS = STEPS_PER_EPOCH / 20
    
config = ShapesConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  64
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  64
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [64 64  3]
INPUT_SHAPE                    [64 64  3]
INPUT_SHAPE_OUTPUT_FEATURE_RATIO 16
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCE

## Training

Load pretrained weights and train in two stages: just heads, then finetune 

In [5]:
model = modellib.DeepLabThreePlus(mode="training", config=config, model_dir=MODEL_DIR)

In [6]:
inititalize_weights_with = "pascal"  # pascal or last

if inititalize_weights_with == "pascal":
    model.load_weights(PASCAL_WEIGHTS_PATH, by_name=True)


### Heads

Train just the heads of the model

In [7]:
model.train(dataset_train, dataset_validate,
            learning_rate=config.LEARNING_RATE,
            epochs=1,
            layers='all')

Instructions for updating:
Use the retry module or similar alternatives.

Starting at epoch 0. LR=0.001

Checkpoint Path: /home/keras/data/shapes/logs/shapes20180517T1930/mask_rcnn_shapes_{epoch:04d}.h5
Selecting layers to train
entry_flow_conv1_1     (Conv2D)
entry_flow_conv1_1_BN   (BatchNormalization)
entry_flow_conv1_2     (Conv2D)
entry_flow_conv1_2_BN   (BatchNormalization)
entry_flow_block1_separable_conv1_depthwise   (DepthwiseConv2D)
entry_flow_block1_separable_conv1_depthwise_BN   (BatchNormalization)
entry_flow_block1_separable_conv1_pointwise   (Conv2D)
entry_flow_block1_separable_conv1_pointwise_BN   (BatchNormalization)
entry_flow_block1_separable_conv2_depthwise   (DepthwiseConv2D)
entry_flow_block1_separable_conv2_depthwise_BN   (BatchNormalization)
entry_flow_block1_separable_conv2_pointwise   (Conv2D)
entry_flow_block1_separable_conv2_pointwise_BN   (BatchNormalization)
entry_flow_block1_separable_conv3_depthwise   (DepthwiseConv2D)
entry_flow_block1_separable_conv3_d

ValueError: No such layer: rpn_class_loss

### Fine Tuning

Fine tune lower layers of the model

## Detection

Run the model on a random image from the test dataset and display the results

In [ ]:
original_image = dataset_train.load_image(1)

In [ ]:
results = model.detect([original_image], verbose=1)

## Evaluation

Evaluate the performance of the model on the full test dataset

In [ ]:
test = ((results[0]+1)*255).astype(np.uint8)

In [ ]:
l1 = test[:,:,1]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
np.shape(l1)

In [ ]:
plt.imshow(original_image)

In [ ]:
plt.imshow(test[:,:,0])

In [ ]:
model.keras_model.layers

In [ ]:
layers = 'heads'

# Pre-defined layer regular expressions
layer_regex = {
    # all layers but the backbone
    "heads": r"(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)",
    # From a specific Resnet stage and up
    "3+": r"(res3.*)|(bn3.*)|(res4.*)|(bn4.*)|(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)",
    "4+": r"(res4.*)|(bn4.*)|(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)",
    "5+": r"(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)",
    # All layers
    "all": ".*",
}
if layers in layer_regex.keys():
    layers = layer_regex[layers]

In [ ]:
layers